Question 1.1-1.5

In [8]:
import pandas as pd
import os

# Helper function to analyze chromosome files
def analyze_chrom_sizes(file_path, species_name):
    df = pd.read_csv(file_path, sep="\t", header=None, names=["chrom", "size"])
    
    total_genome_size = df["size"].sum()
    num_chromosomes = len(df)
    largest_chrom = df.loc[df["size"].idxmax()]
    smallest_chrom = df.loc[df["size"].idxmin()]
    mean_size = df["size"].mean()
    
    return {
        "Species": species_name,
        "Total genome size (bp)": total_genome_size,
        "Number of chromosomes": num_chromosomes,
        "Largest chromosome": largest_chrom["chrom"],
        "Largest size (bp)": largest_chrom["size"],
        "Smallest chromosome": smallest_chrom["chrom"],
        "Smallest size (bp)": smallest_chrom["size"],
        "Mean size (bp)": mean_size
    }

In [10]:
# define file path
files = {
    "E. coli K12": os.path.join("ecoli.chrom.sizes"),
    "Yeast (sacCer3)": os.path.join("yeast.chrom.sizes"),
    "Fruit Fly (dm6)": os.path.join("dm6.chrom.sizes"),
    "Arabidopsis (TAIR10)": os.path.join("TAIR10.chrom.sizes"),
    "Tomato (v4.00)": os.path.join("tomato.chrom.sizes"),
    "Human (hg38)": os.path.join("hg38.chrom.sizes"),
    "Wheat (IWGSC)": os.path.join("wheat.chrom.sizes"),
}

In [11]:
# run analysis 
results = []
for species, path in files.items():
    res = analyze_chrom_sizes(path, species)
    results.append(res)

summary_table = pd.DataFrame(results)
summary_table

,Species,Total genome size (bp),Number of chromosomes,Largest chromosome,Largest size (bp),Smallest chromosome,Smallest size (bp),Mean size (bp)
0,E. coli K12,4639211,1,Ecoli,4639211,Ecoli,4639211,4.639211e+06
1,Yeast (sacCer3),12157105,17,chrIV,1531933,chrM,85779,7.151238e+05
2,Fruit Fly (dm6),137547960,7,chr3R,32079331,chr4,1348131,1.964971e+07
3,Arabidopsis (TAIR10),119146348,5,Chr1,30427671,Chr4,18585056,2.382927e+07
4,Tomato (v4.00),782520033,13,ch01,90863682,ch00,9643250,6.019385e+07
5,Human (hg38),3088269832,24,chr1,248956422,chr21,46709983,1.286779e+08
6,Wheat (IWGSC),14547261565,22,3B,830829764,6D,473592718,6.612392e+08


Question 2.1

Question 2.2-2.5
helper functions

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Poisson pmf 
def poisson_pmf_vector(k_vals, lam):
    k_vals = np.asarray(k_vals, dtype=float)
    # pmf(k) = exp( k*log(lam) - lam - lgamma(k+1) )
    return np.exp(k_vals*np.log(lam) - lam - np.array([math.lgamma(k+1) for k in k_vals]))

# Normal bin masses 
def normal_bin_masses(k_vals, mu, sigma):
    # mass for integer bin k is Phi(k+0.5) - Phi(k-0.5)
    def Phi(x):
        return 0.5*(1.0 + math.erf((x - mu)/(sigma*np.sqrt(2))))
    k_vals = np.asarray(k_vals, dtype=float)
    left  = np.array([Phi(k - 0.5) for k in k_vals])
    right = np.array([Phi(k + 0.5) for k in k_vals])
    return np.clip(right - left, 0.0, 1.0)

# simulator
def simulate_coverage(genome_size=1_000_000, read_len=100, coverage=3, rng_seed=42):
    """
    Returns:
      coverage_array: length `genome_size`, number of times each base was covered
      num_reads: number of simulated reads actually placed
    """
    rng = np.random.default_rng(rng_seed)
    num_reads = int(round(coverage * genome_size / read_len))
    cov = np.zeros(genome_size, dtype=np.int32)

    # Uniform start in [0, genome_size - read_len]
    starts = rng.integers(low=0, high=genome_size - read_len + 1, size=num_reads)
    # Add coverage for each read; vectorized via cumulative trick
    # We'll mark +1 at start and -1 just after end, then cumsum.
    diff = np.zeros(genome_size + 1, dtype=np.int32)
    diff[starts] += 1
    ends = starts + read_len  # exclusive
    diff[ends]  -= 1
    cov = np.cumsum(diff[:-1])

    return cov, num_reads

# Histogram + overlays 
def plot_hist_with_overlays(cov, lam, title, normal_sigma=None, max_k=None):
    genome_size = len(cov)
    # histogram of per-base coverage (counts of bases with k coverage)
    counts = np.bincount(cov)  # index = k, value = how many bases have coverage k

    # choose x-range
    if max_k is None:
        # show up to, say, mean + 6*sqrt(mean), but not beyond observed max
        obs_max = np.argmax(counts[::-1] > 0)
        obs_max = len(counts) - 1 - obs_max  # last nonzero bin
        max_k = int(min(obs_max, lam + 6*math.sqrt(lam)))
    k = np.arange(0, max_k + 1, dtype=int)

    # observed histogram (align bars at integers)
    plt.figure(figsize=(8,5))
    plt.bar(k, counts[:len(k)], width=0.9, alpha=0.6, label="Observed (simulation)")

    # Poisson overlay (expected counts per k)
    pois = poisson_pmf_vector(k, lam) * genome_size
    plt.plot(k, pois, linewidth=2, label=f"Poisson λ={lam} (expected)")

    # Normal overlay (discretized to integer bins)
    if normal_sigma is None:
        normal_sigma = math.sqrt(lam)
    norm_masses = normal_bin_masses(k, mu=lam, sigma=normal_sigma) * genome_size
    plt.plot(k, norm_masses, linewidth=2, linestyle="--",
             label=f"Normal μ={lam}, σ={normal_sigma:.2f} (expected)")

    plt.title(title)
    plt.xlabel("Per-base coverage (k)")
    plt.ylabel("Number of bases with coverage k")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return counts

# reporting helper
def report_zero_coverage(counts, lam, genome_size):
    observed_zero = int(counts[0]) if len(counts) > 0 else 0
    expected_zero = int(round(genome_size * math.exp(-lam)))
    frac_obs = observed_zero / genome_size
    frac_exp = math.exp(-lam)
    print(f"Zero-coverage bases: observed = {observed_zero:,} "
          f"({frac_obs:.4%}), expected Poisson = {expected_zero:,} ({frac_exp:.4%})")

Question 2.2 (3x coverage)

In [ ]:
G = 1_000_000
L = 100
C = 3
cov3, nreads3 = simulate_coverage(genome_size=G, read_len=L, coverage=C, rng_seed=123)
print(f"Simulated reads (should be ~{int(C*G/L):,}): {nreads3:,}")

counts3 = plot_hist_with_overlays(
    cov3, lam=C, title="Coverage histogram @ 3×",
    normal_sigma=math.sqrt(3),  # 1.732...
    max_k=None
)

report_zero_coverage(counts3, lam=C, genome_size=G)

Question 2.3

Question 2.4 (10x coverage)

In [ ]:
C = 10
cov10, nreads10 = simulate_coverage(genome_size=G, read_len=L, coverage=C, rng_seed=123)
print(f"Simulated reads (should be ~{int(C*G/L):,}): {nreads10:,}")

counts10 = plot_hist_with_overlays(
    cov10, lam=C, title="Coverage histogram @ 10×",
    normal_sigma=3.16,  # given
    max_k=None
)

report_zero_coverage(counts10, lam=C, genome_size=G)

Question 2.5 (30x coverage)

In [ ]:
C = 30
cov30, nreads30 = simulate_coverage(genome_size=G, read_len=L, coverage=C, rng_seed=123)
print(f"Simulated reads (should be ~{int(C*G/L):,}): {nreads30:,}")

counts30 = plot_hist_with_overlays(
    cov30, lam=C, title="Coverage histogram @ 30×",
    normal_sigma=5.47,  # given
    max_k=None
)

report_zero_coverage(counts30, lam=C, genome_size=G)